In [14]:
print(X_train.shape)  # Should match input_size


torch.Size([2395, 15])


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import flwr as fl
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from torch.utils.data import DataLoader, TensorDataset

# Load dataset
df = pd.read_csv("young_adults.csv")
X = df.iloc[:, :-1].values  # Features
y = df.iloc[:, -1].values   # Target

# Balance dataset using SMOTE
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=64)

# Define MLP Model
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return torch.sigmoid(self.fc3(x))



# Initialize model
input_size = X_train.shape[1]  
model = MLP(input_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.00545)

# Define Flower client, always labeled as Client 1
class FLClient(fl.client.NumPyClient):
    def __init__(self):
        self.cid = 1  # Always set this client as Client 1

    def get_parameters(self, config):
        print(f"[Client {self.cid}] Sending model parameters")
        return [val.cpu().detach().numpy() for val in model.state_dict().values()]
    
    def set_parameters(self, parameters):
        print(f"[Client {self.cid}] Receiving model parameters")
        param_dict = zip(model.state_dict().keys(), parameters)
        state_dict = {k: torch.tensor(v) for k, v in param_dict}
        model.load_state_dict(state_dict, strict=True)
    
    def fit(self, parameters, config):
        self.set_parameters(parameters)
        model.train()
        for epoch in range(10):
            total_loss = 0
            for X_batch, y_batch in train_loader:
                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            print(f"[Client {self.cid}] Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")
        return self.get_parameters(config), len(X_train), {}
    
    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        model.eval()
        correct, total, loss = 0, 0, 0.0
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                outputs = model(X_batch)
                loss += criterion(outputs, y_batch).item()
                predictions = (outputs > 0.5).float()
                correct += (predictions == y_batch).sum().item()
                total += y_batch.size(0)
        
        accuracy = correct / total if total > 0 else 0.0
        print(f"[Client {self.cid}] Accuracy: {accuracy:.4f}")
        return float(loss), len(X_test), {"accuracy": accuracy}

# Start the client (Always as Client 1)
fl.client.start_numpy_client(server_address="127.0.0.1:8080", client=FLClient())


	Instead, use `flwr.client.start_client()` by ensuring you first call the `.to_client()` method as shown below: 
	flwr.client.start_client(
		server_address='<IP>:<PORT>',
		client=FlowerClient().to_client(), # <-- where FlowerClient is of type flwr.client.NumPyClient object
	)
	Using `start_numpy_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        


_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:8080: ConnectEx: Connection refused (No connection could be made because the target machine actively refused it.
 -- 10061)"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:8080: ConnectEx: Connection refused (No connection could be made because the target machine actively refused it.\r\n -- 10061)", grpc_status:14, created_time:"2025-03-17T09:58:49.0930157+00:00"}"
>

In [ ]:
today

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import flwr as fl
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from torch.utils.data import DataLoader, TensorDataset
import socket

# Load dataset
df = pd.read_csv("young_adults.csv")
X = df.iloc[:, :-1].values  
y = df.iloc[:, -1].values  

# Balance dataset using SMOTE
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoaders
batch_size = 128
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=batch_size, shuffle=False)

# Define the Fine-Tuned MLP Model
class MLP(nn.Module):
    def __init__(self, input_size, hidden1=512, hidden2=512, hidden3=256, hidden4=128, output_size=2, dropout_prob=0.2):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1)
        self.bn1 = nn.BatchNorm1d(hidden1)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_prob)

        self.fc2 = nn.Linear(hidden1, hidden2)
        self.bn2 = nn.BatchNorm1d(hidden2)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_prob)

        self.fc3 = nn.Linear(hidden2, hidden3)
        self.bn3 = nn.BatchNorm1d(hidden3)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(dropout_prob)

        self.fc4 = nn.Linear(hidden3, hidden4)
        self.bn4 = nn.BatchNorm1d(hidden4)
        self.relu4 = nn.ReLU()
        self.dropout4 = nn.Dropout(dropout_prob)

        self.fc5 = nn.Linear(hidden4, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.dropout2(x)

        x = self.fc3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.dropout3(x)

        x = self.fc4(x)
        x = self.bn4(x)
        x = self.relu4(x)
        x = self.dropout4(x)

        x = self.fc5(x)
        return x

# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size = X_train.shape[1]  
model = MLP(input_size).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0008, weight_decay=5e-4)

# Learning Rate Scheduler
scheduler = torch.optim.lr_scheduler.CyclicLR(
    optimizer, 
    base_lr=0.0001,  
    max_lr=0.001,  
    step_size_up=len(train_loader) * 10,  
    mode='triangular'
)

scheduler2 = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=20)

# Define Flower Client
class FLClient(fl.client.NumPyClient):
    def __init__(self):
        self.cid = 1  

    def wait_for_server_ready(self, host="127.0.0.1", port=8081):
        """Wait for the aggregation server to signal readiness before starting training."""
        print("🔄 Waiting for the aggregation server to be ready...")
        
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            try:
                sock.connect((host, port))
                sock.sendall(b"PONG")  # Respond to PING
                msg = sock.recv(1024)  # Wait for READY signal

                if msg.strip() == b"READY":
                    print("✅ Aggregation server is ready! Sending ACK...")
                    sock.sendall(b"ACK")  # Confirm readiness
                else:
                    print("❌ Did not receive READY signal. Exiting.")
                    exit(1)

            except ConnectionRefusedError:
                print("❌ Could not connect to the aggregation server. Exiting.")
                exit(1)

    def fit(self, parameters, config):
        """Train the model using received parameters."""
        self.set_parameters(parameters)

        print(f"[Client {self.cid}] Starting training...")
        model.train()

        for epoch in range(10):
            total_loss = 0
            for X_batch, y_batch in train_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)

                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
            
            print(f"[Client {self.cid}] Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

        return self.get_parameters(config), len(X_train), {}

# Before starting training, wait for the server to be ready
client = FLClient()
client.wait_for_server_ready()

# Start the client
fl.client.start_numpy_client(server_address="127.0.0.1:8081", client=client)


	Instead, use `flwr.client.start_client()` by ensuring you first call the `.to_client()` method as shown below: 
	flwr.client.start_client(
		server_address='<IP>:<PORT>',
		client=FlowerClient().to_client(), # <-- where FlowerClient is of type flwr.client.NumPyClient object
	)
	Using `start_numpy_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        


🔄 Waiting for the aggregation server to be ready...
❌ Did not receive READY signal. Exiting.


_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:8081: End of TCP stream"
	debug_error_string = "UNKNOWN:Error received from peer  {created_time:"2025-03-17T09:58:52.9990809+00:00", grpc_status:14, grpc_message:"failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:8081: End of TCP stream"}"
>

cient 1 form helthsheild


In [1]:
import torch
import torch.nn as nn
import flwr as fl
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import StepLR
import opacus
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import time
import os
import json
from typing import Dict, List, Tuple

In [ ]:
import torch
import torch.nn as nn
import flwr as fl
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import StepLR
from sklearn.preprocessing import StandardScaler
import logging
import traceback
import os
import argparse
import time
from typing import Dict, List, Tuple

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("client_debug.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger("FL-Client")

# CLIENT CONFIGURATION
CLIENT_CONFIG = {
    "local_epochs": 3,
    "batch_size": 32,
    "learning_rate": 0.001,
    "weight_decay": 1e-5,
    "dropout_rate": 0.3,
    "fedprox_mu": 0.01,
}

# MODEL DEFINITION
class HealthcareModel(nn.Module):
    def __init__(self, input_size=15, dropout_rate=0.3):
        super(HealthcareModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

# LOAD SYNTHETIC DATA
def generate_synthetic_health_data(num_samples=300, binary_label_ratio=0.2):
    """Generate synthetic healthcare data for simulation"""
    np.random.seed(42)  # For reproducibility
    
    # Create features
    age = np.random.normal(50, 15, num_samples)
    bmi = np.random.normal(28, 7, num_samples)
    blood_glucose = np.random.normal(100, 25, num_samples)
    systolic_bp = np.random.normal(130, 20, num_samples)
    diastolic_bp = np.random.normal(80, 10, num_samples)
    hdl_cholesterol = np.random.normal(50, 15, num_samples)
    ldl_cholesterol = np.random.normal(100, 35, num_samples)
    triglycerides = np.random.normal(150, 80, num_samples)
    heart_rate = np.random.normal(70, 10, num_samples)
    
    # Binary features
    is_male = np.random.binomial(1, 0.5, num_samples)
    smoker = np.random.binomial(1, 0.3, num_samples)
    on_bp_meds = np.random.binomial(1, 0.25, num_samples)
    has_hypertension = np.random.binomial(1, 0.3, num_samples)
    has_diabetes = np.random.binomial(1, 0.15, num_samples)
    family_history = np.random.binomial(1, 0.4, num_samples)
    
    # Combine features
    X = np.column_stack([
        is_male, age, smoker, on_bp_meds, has_hypertension, has_diabetes,
        family_history, bmi, blood_glucose, systolic_bp, diastolic_bp,
        hdl_cholesterol, ldl_cholesterol, triglycerides, heart_rate
    ])
    
    # Generate target (binary classification)
    # Use a simplified model where higher age, being male, smoking, having 
    # hypertension, diabetes, high BMI, and family history increase risk
    risk_score = (
        0.1 * age/100 + 
        0.05 * is_male +
        0.1 * smoker + 
        0.1 * has_hypertension +
        0.15 * has_diabetes +
        0.1 * (bmi > 30).astype(int) +
        0.1 * (blood_glucose > 125).astype(int) +
        0.1 * (systolic_bp > 140).astype(int) +
        0.05 * (hdl_cholesterol < 40).astype(int) +
        0.05 * (ldl_cholesterol > 130).astype(int) +
        0.05 * family_history +
        np.random.normal(0, 0.1, num_samples)  # Add some noise
    )
    
    # Convert to binary outcome (CHD)
    threshold = np.percentile(risk_score, 100 - binary_label_ratio*100)
    y = (risk_score >= threshold).astype(int)
    
    # Create column names for the dataframe
    feature_names = [
        'male', 'age', 'currentSmoker', 'BPMeds', 'prevalentHyp', 'diabetes',
        'familyHistory', 'BMI', 'glucose', 'sysBP', 'diaBP',
        'HDL', 'LDL', 'triglycerides', 'heartRate'
    ]
    
    # Create dataframe
    df = pd.DataFrame(X, columns=feature_names)
    df['TenYearCHD'] = y
    
    print(f"Generated synthetic dataset with {num_samples} samples")
    print(f"Class distribution: {df['TenYearCHD'].value_counts().to_dict()}")
    
    return df

def load_and_preprocess_data(client_id):
    """Load data (synthetic or from file) and preprocess it"""
    try:
        # Try to load from CSV if available
        if os.path.exists(f"client_{client_id}_data.csv"):
            print(f"Loading dataset for client {client_id} from CSV...")
            df = pd.read_csv(f"client_{client_id}_data.csv")
        else:
            # Otherwise generate synthetic data
            print(f"Generating synthetic data for client {client_id}...")
            df = generate_synthetic_health_data(num_samples=300)
            # Save for future use
            df.to_csv(f"client_{client_id}_data.csv", index=False)
        
        print(f"Dataset shape: {df.shape}")
        
        # Check for missing values
        missing_values = df.isnull().sum().sum()
        if missing_values > 0:
            print(f"Missing values: {missing_values}, dropping rows...")
            df.dropna(inplace=True)
        
        # Separate features and label
        X = df.drop(columns=["TenYearCHD"])
        y = df["TenYearCHD"]
        
        # Print class distribution
        print(f"Class distribution: {y.value_counts().to_dict()}")
        
        # Normalize features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # Convert to tensors
        X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
        y_tensor = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)
        
        print(f"Data processed: X shape {X_tensor.shape}, y shape {y_tensor.shape}")
        return X_tensor, y_tensor
    
    except Exception as e:
        logger.error(f"Error loading data: {str(e)}")
        logger.error(traceback.format_exc())
        raise

# FEDPROX LOSS
class FedProxLoss(nn.Module):
    def __init__(self, mu=0.01):
        super(FedProxLoss, self).__init__()
        self.mu = mu
        self.base_loss = nn.BCELoss()

    def forward(self, preds, labels, local_params, global_params):
        # Handle basic BCE loss
        loss = self.base_loss(preds, labels)
        
        # Add FedProx regularization term if global parameters exist
        if global_params is not None and len(global_params) > 0:
            try:
                # Calculate proximal term
                proximal_term = 0.0
                for local_param, global_param in zip(local_params, global_params):
                    if local_param.shape != global_param.shape:
                        logger.warning(f"Shape mismatch: local {local_param.shape}, global {global_param.shape}")
                        continue
                        
                    param_diff = local_param - global_param
                    proximal_term += torch.sum(param_diff ** 2)
                
                # Add weighted proximal term to loss
                return loss + (self.mu / 2) * proximal_term
            except Exception as e:
                logger.error(f"Error in FedProx calculation: {str(e)}")
                logger.error(traceback.format_exc())
                # Fall back to regular loss if proximal term fails
                return loss
                
        return loss

# CLIENT
class HealthcareClient(fl.client.NumPyClient):
    def __init__(self, client_id, model, train_loader, device, verbose=True):
        self.client_id = client_id
        self.model = model
        self.train_loader = train_loader
        self.device = device
        self.global_params = None
        self.rounds_completed = 0
        self.verbose = verbose
        logger.info(f"Initialized client {client_id} with device: {device}")

    def get_parameters(self, config):
        try:
            params = [val.cpu().detach().numpy() for val in self.model.parameters()]
            if self.verbose:
                logger.info(f"Client {self.client_id}: Returning {len(params)} parameter arrays")
            return params
        except Exception as e:
            logger.error(f"Error in get_parameters: {str(e)}")
            logger.error(traceback.format_exc())
            raise

    def set_parameters(self, parameters):
        try:
            logger.info(f"Client {self.client_id}: Setting {len(parameters)} parameter arrays")
            
            # Store global parameters for FedProx
            self.global_params = [torch.tensor(p).to(self.device) for p in parameters]
            
            # Update model parameters
            for param, param_data in zip(self.model.parameters(), self.global_params):
                param.data.copy_(param_data)
                
            if self.verbose:
                logger.info(f"Client {self.client_id}: Parameters successfully updated")
        except Exception as e:
            logger.error(f"Error in set_parameters: {str(e)}")
            logger.error(traceback.format_exc())
            raise

    def fit(self, parameters, config):
        try:
            # Set parameters from server
            self.set_parameters(parameters)
            logger.info(f"Client {self.client_id}: Starting training")
            
            # Set model to training mode
            self.model.train()
            
            # Initialize optimizer and loss function
            optimizer = torch.optim.Adam(
                self.model.parameters(), 
                lr=CLIENT_CONFIG["learning_rate"], 
                weight_decay=CLIENT_CONFIG["weight_decay"]
            )
            
            scheduler = StepLR(optimizer, step_size=2, gamma=0.9)
            criterion = FedProxLoss(mu=CLIENT_CONFIG["fedprox_mu"])
            
            # Track metrics
            total_loss = 0.0
            correct = 0
            total = 0
            
            # Training loop
            for epoch in range(CLIENT_CONFIG["local_epochs"]):
                epoch_loss = 0.0
                epoch_samples = 0
                
                for batch_idx, (X_batch, y_batch) in enumerate(self.train_loader):
                    # Move data to device
                    X_batch, y_batch = X_batch.to(self.device), y_batch.to(self.device)
                    
                    # Forward pass
                    optimizer.zero_grad()
                    y_pred = self.model(X_batch)
                    
                    # Adjust dimensions if needed
                    y_pred = y_pred.squeeze()
                    y_batch = y_batch.squeeze()
                    if y_pred.shape != y_batch.shape:
                        y_batch = y_batch.view_as(y_pred)
                    
                    # Calculate loss
                    loss = criterion(y_pred, y_batch, self.model.parameters(), self.global_params)
                    
                    # Backward pass and optimize
                    loss.backward()
                    optimizer.step()
                    
                    # Update metrics
                    current_loss = loss.item() * X_batch.size(0)
                    total_loss += current_loss
                    epoch_loss += current_loss
                    epoch_samples += X_batch.size(0)
                    
                    # Calculate accuracy
                    predicted = (y_pred > 0.5).float()
                    batch_correct = (predicted == y_batch).sum().item()
                    correct += batch_correct
                    total += y_batch.size(0)
                    
                    if self.verbose and batch_idx % 5 == 0:
                        logger.info(f"Client {self.client_id}: Epoch {epoch+1}/{CLIENT_CONFIG['local_epochs']}, "
                                    f"Batch {batch_idx}/{len(self.train_loader)}, "
                                    f"Loss: {current_loss/X_batch.size(0):.4f}, "
                                    f"Batch Accuracy: {batch_correct/y_batch.size(0):.4f}")
                
                # Step scheduler after each epoch
                scheduler.step()
                
                # Log epoch metrics
                avg_epoch_loss = epoch_loss / epoch_samples if epoch_samples > 0 else 0
                logger.info(f"Client {self.client_id}: Epoch {epoch+1}/{CLIENT_CONFIG['local_epochs']} completed, "
                            f"Loss: {avg_epoch_loss:.4f}")
            
            # Calculate final metrics
            accuracy = correct / total if total > 0 else 0
            avg_loss = total_loss / total if total > 0 else 0
            
            # Increment round counter
            self.rounds_completed += 1
            
            logger.info(f"Client {self.client_id}: Training completed (Round {self.rounds_completed}), "
                        f"Accuracy: {accuracy:.4f}, Loss: {avg_loss:.4f}")
            
            # Return updated model parameters, sample count, and metrics
            return self.get_parameters(config), total, {"accuracy": float(accuracy), "loss": float(avg_loss)}
        
        except Exception as e:
            logger.error(f"Error during fit: {str(e)}")
            logger.error(traceback.format_exc())
            raise

    def evaluate(self, parameters, config):
        try:
            # Set parameters from server
            self.set_parameters(parameters)
            
            # Set model to evaluation mode
            self.model.eval()
            
            # Initialize metrics
            loss = 0.0
            correct = 0
            total = 0
            criterion = nn.BCELoss()
            
            # Evaluation loop
            with torch.no_grad():
                for X_batch, y_batch in self.train_loader:
                    # Move data to device
                    X_batch, y_batch = X_batch.to(self.device), y_batch.to(self.device)
                    
                    # Forward pass
                    y_pred = self.model(X_batch).squeeze()
                    
                    # Adjust dimensions if needed
                    y_batch = y_batch.squeeze()
                    if y_pred.shape != y_batch.shape:
                        y_batch = y_batch.view_as(y_pred)
                    
                    # Calculate loss
                    batch_loss = criterion(y_pred, y_batch).item() * y_batch.size(0)
                    loss += batch_loss
                    
                    # Calculate accuracy
                    predicted = (y_pred > 0.5).float()
                    correct += (predicted == y_batch).sum().item()
                    total += y_batch.size(0)
            
            # Calculate final metrics
            avg_loss = loss / total if total > 0 else 0
            accuracy = correct / total if total > 0 else 0
            
            if self.verbose:
                logger.info(f"Client {self.client_id}: Evaluation completed, "
                        f"Accuracy: {accuracy:.4f}, Loss: {avg_loss:.4f}")
            
            # Return metrics
            return float(avg_loss), total, {"accuracy": float(accuracy), "loss": float(avg_loss)}
        
        except Exception as e:
            logger.error(f"Error during evaluate: {str(e)}")
            logger.error(traceback.format_exc())
            raise

def start_client(client_id=0, port=8080, host="localhost", verbose=True):
    try:
        # Set device for computation
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Client {client_id} using device: {device}")
        
        # Load and preprocess data
        X_tensor, y_tensor = load_and_preprocess_data(client_id)
        
        # Create dataset and dataloader
        dataset = TensorDataset(X_tensor, y_tensor)
        train_loader = DataLoader(dataset, batch_size=CLIENT_CONFIG["batch_size"], shuffle=True)
        
        # Initialize model
        input_size = X_tensor.shape[1]
        print(f"Initializing model with input size: {input_size}")
        model = HealthcareModel(input_size=input_size, dropout_rate=CLIENT_CONFIG["dropout_rate"]).to(device)
        
        # Print model summary if verbose
        if verbose:
            print(model)
        
        # Initialize client
        client = HealthcareClient(client_id, model, train_loader, device, verbose)
        
        # Start client and connect to server
        server_address = f"{host}:{port}"
        print(f"Client {client_id} connecting to server at {server_address}")
        
        # Convert to Client and start
        fl.client.start_client(server_address=server_address, client=client)
        
    except Exception as e:
        logger.error(f"Error in client {client_id}: {str(e)}")
        logger.error(traceback.format_exc())
        print(f"Client {client_id} failed with error: {str(e)}")

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Start a Federated Learning client")
    parser.add_argument("--id", type=int, default=0, help="Client ID")
    parser.add_argument("--port", type=int, default=8080, help="Server port")
    parser.add_argument("--host", type=str, default="localhost", help="Server hostname")
    parser.add_argument("--verbose", action="store_true", help="Enable verbose output")
    
    args = parser.parse_args()
    
    start_client(client_id=args.id, port=args.port, host=args.host, verbose=args.verbose)

	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
05/09/2025 22:43:43:WARNING:DEPRECATED FEATURE: flwr.client.start_client() is deprecated.
	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
05/09/2025 22:43:43:DEBUG:Opened insecure gRPC connection (no certificates were passed)
05/09/2025 22:43:43:DEBUG:ChannelConnectivity.IDLE
05/09/2025 22:43:43:DEB

Using device: cpu
Loading dataset from CSV...
Original dataset shape: (1060, 16)
Missing values before cleaning: 0
Dataset shape after removing missing values: (1060, 16)
Class distribution: {0: 884, 1: 176}
Feature columns: ['male', 'age', 'education', 'currentSmoker', 'cigsPerDay', 'BPMeds', 'prevalentStroke', 'prevalentHyp', 'diabetes', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose']
Data loaded: X shape torch.Size([1060, 15]), y shape torch.Size([1060, 1])
Initializing model with input size: 15
HealthcareModel(
  (layers): Sequential(
    (0): Linear(in_features=15, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=256, out_features=128, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=128, out_features=64, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.3, inplace=False)
    (9): Linear(in_features=64, out_features=1, bias=True)
    (10): Sigmoid()
  )
)
Starting 

INFO :      
05/09/2025 22:43:45:INFO:
INFO :      Received: evaluate message 1fa62300-869b-4b00-b2b0-f65f8386a52f
05/09/2025 22:43:45:INFO:Received: evaluate message 1fa62300-869b-4b00-b2b0-f65f8386a52f
05/09/2025 22:43:45:WARNING:Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
INFO :      Sent reply
05/09/2025 22:43:45:INFO:Sent reply
INFO :      
05/09/2025 22:43:45:INFO:
INFO :      Received: evaluate message de2dbff1-770d-46a6-ab4a-294fc8356de1
05/09/2025 22:43:45:INFO:Received: evaluate message de2dbff1-770d-46a6-ab4a-294fc8356de1
05/09/2025 22:43:45:WARNING:Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
INFO :      Sent reply
05/09/2025 22:43:45:INFO:Sent reply
INFO :      
05/

In [1]:
import torch
import torch.nn as nn
import flwr as fl
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
import os
import logging
import argparse


In [6]:
import torch
import torch.nn as nn
import flwr as fl
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
import os
import sys
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger("FraminghamClient")

# Client configuration
CLIENT_CONFIG = {
    "local_epochs": 3,
    "batch_size": 32,
    "learning_rate": 0.001,
    "weight_decay": 1e-5,
    "dropout_rate": 0.3,
    "server_address": "localhost:8080"
}

# Model for Framingham Heart Study data
class HeartDiseaseModel(nn.Module):
    def __init__(self, input_size):
        super(HeartDiseaseModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Dropout(CLIENT_CONFIG["dropout_rate"]),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(CLIENT_CONFIG["dropout_rate"]),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

# Load and preprocess Framingham data
def load_data(data_path):
    """Load and preprocess Framingham Heart Study data"""
    try:
        # Read CSV data
        df = pd.read_csv(data_path)
        logger.info(f"Loaded {data_path} with shape {df.shape}")
        
        # Check for missing values
        missing_values = df.isnull().sum().sum()
        if missing_values > 0:
            logger.info(f"Found {missing_values} missing values, dropping rows with missing values")
            df.dropna(inplace=True)
            logger.info(f"Shape after dropping missing values: {df.shape}")
        
        # Ensure the target column exists
        if "TenYearCHD" not in df.columns:
            raise ValueError("Target column 'TenYearCHD' not found in dataset!")
            
        # Split features and target
        X = df.drop(columns=["TenYearCHD"])
        y = df["TenYearCHD"]
        
        # Show class distribution
        logger.info(f"Class distribution: {y.value_counts().to_dict()}")
        
        # Standardize features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # Convert to tensors
        X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
        y_tensor = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)
        
        # Create dataset and dataloader
        dataset = TensorDataset(X_tensor, y_tensor)
        dataloader = DataLoader(dataset, batch_size=CLIENT_CONFIG["batch_size"], shuffle=True)
        
        logger.info(f"Created dataloader with {len(dataset)} samples and {X.shape[1]} features")
        return dataloader, X.shape[1]
    
    except Exception as e:
        logger.error(f"Error loading data: {str(e)}")
        raise

# Client class for Federated Learning
class FraminghamClient(fl.client.NumPyClient):
    def __init__(self, model, dataloader, device):
        self.model = model
        self.dataloader = dataloader
        self.device = device
        logger.info(f"Initialized client with device: {device}")
        
    def get_parameters(self, config):
        """Get model parameters as a list of NumPy arrays"""
        # Using detach() to prevent gradient error
        return [val.detach().cpu().numpy() for val in self.model.parameters()]
    
    def set_parameters(self, parameters):
        """Set model parameters from a list of NumPy arrays"""
        params_dict = zip(self.model.state_dict().keys(), parameters)
        state_dict = {k: torch.tensor(v) for k, v in params_dict}
        self.model.load_state_dict(state_dict, strict=True)
        logger.info("Parameters updated from server")
        
    def fit(self, parameters, config):
        """Train the model on local data"""
        # Update model with server parameters
        self.set_parameters(parameters)
        
        # Train the model
        self.model.train()
        optimizer = torch.optim.Adam(
            self.model.parameters(), 
            lr=CLIENT_CONFIG["learning_rate"],
            weight_decay=CLIENT_CONFIG["weight_decay"]
        )
        criterion = nn.BCELoss()
        
        # Metrics for tracking
        total_loss = 0.0
        total_samples = 0
        correct = 0
        
        # Train for multiple epochs
        for epoch in range(CLIENT_CONFIG["local_epochs"]):
            epoch_loss = 0.0
            epoch_samples = 0
            
            for batch_idx, (X, y) in enumerate(self.dataloader):
                # Move tensors to device
                X, y = X.to(self.device), y.to(self.device)
                
                # Forward pass
                y_pred = self.model(X)
                loss = criterion(y_pred, y)
                
                # Backward pass
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                # Update metrics
                batch_loss = loss.item() * X.size(0)
                total_loss += batch_loss
                epoch_loss += batch_loss
                total_samples += X.size(0)
                epoch_samples += X.size(0)
                
                # Calculate accuracy
                predicted = (y_pred > 0.5).float()
                correct += (predicted == y).sum().item()
                
                # Log progress occasionally
                if batch_idx % 5 == 0:
                    logger.info(
                        f"Epoch {epoch+1}/{CLIENT_CONFIG['local_epochs']} - "
                        f"Batch {batch_idx}/{len(self.dataloader)} - "
                        f"Loss: {loss.item():.4f}"
                    )
            
            # Log epoch metrics
            logger.info(
                f"Epoch {epoch+1}/{CLIENT_CONFIG['local_epochs']} completed - "
                f"Loss: {epoch_loss/epoch_samples:.4f}"
            )
        
        # Calculate final metrics
        avg_loss = total_loss / total_samples if total_samples > 0 else 0
        accuracy = correct / total_samples if total_samples > 0 else 0
        
        logger.info(f"Training completed - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")
        
        # Return updated model parameters and metrics
        return self.get_parameters({}), total_samples, {"loss": float(avg_loss), "accuracy": float(accuracy)}
    
    def evaluate(self, parameters, config):
        """Evaluate the model on local data"""
        # Update model with server parameters
        self.set_parameters(parameters)
        
        # Evaluate the model
        self.model.eval()
        criterion = nn.BCELoss()
        
        loss = 0.0
        total = 0
        correct = 0
        
        with torch.no_grad():
            for X, y in self.dataloader:
                # Move tensors to device
                X, y = X.to(self.device), y.to(self.device)
                
                # Forward pass
                y_pred = self.model(X)
                batch_loss = criterion(y_pred, y).item()
                
                # Update metrics
                loss += batch_loss * X.size(0)
                total += X.size(0)
                
                # Calculate accuracy
                predicted = (y_pred > 0.5).float()
                correct += (predicted == y).sum().item()
        
        # Calculate final metrics
        avg_loss = loss / total if total > 0 else 0
        accuracy = correct / total if total > 0 else 0
        
        logger.info(f"Evaluation - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")
        
        # Return metrics
        return float(avg_loss), total, {"accuracy": float(accuracy)}

def start_client(client_id=0, server_address=None):
    """Initialize and start a client"""
    # Update server address if provided
    if server_address:
        CLIENT_CONFIG["server_address"] = server_address
    
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logger.info(f"Using device: {device}")
    
    # Determine which data file to use based on client ID
    data_path = f"framingham_part{client_id+1}.csv"
    
    # Try alternative naming if file doesn't exist
    if not os.path.exists(data_path):
        alternative_path = f"framingham_part{client_id+1}.csv"
        if os.path.exists(alternative_path):
            data_path = alternative_path
        else:
            logger.error(f"Data file {data_path} not found")
            return
    
    # Load data
    dataloader, input_size = load_data(data_path)
    
    # Initialize model
    model = HeartDiseaseModel(input_size=input_size).to(device)
    logger.info(f"Model initialized with input size: {input_size}")
    
    # Create client
    client = FraminghamClient(model, dataloader, device)
    
    # Start client
    logger.info(f"Starting client {client_id} and connecting to {CLIENT_CONFIG['server_address']}")
    
    print(f"\n===== Framingham Heart Study FL Client {client_id} =====")
    print(f"Server:        {CLIENT_CONFIG['server_address']}")
    print(f"Data file:     {data_path}")
    print(f"Local epochs:  {CLIENT_CONFIG['local_epochs']}")
    print(f"Batch size:    {CLIENT_CONFIG['batch_size']}")
    print(f"Device:        {device}")
    print("==============================================")
    print(f"\nConnecting to server...\n")
    
    fl.client.start_client(server_address=CLIENT_CONFIG["server_address"], client=client)

# For Jupyter usage
if __name__ == "__main__":
    # Check if running in Jupyter
    if 'ipykernel' in sys.modules:
        print("Running in Jupyter/IPython environment")
        # Default to client ID 0, can be changed by user
        start_client(client_id=0)
    else:
        # For command line use
        import argparse
        parser = argparse.ArgumentParser(description="Framingham Heart Study FL Client")
        parser.add_argument("--id", type=int, default=0, help="Client ID (0, 1, or 2)")
        parser.add_argument("--server", type=str, default="localhost:8080", help="Server address")
        
        args = parser.parse_args()
        
        if args.id not in [0, 1, 2]:
            logger.error("Client ID must be 0, 1, or 2")
        else:
            try:
                start_client(args.id, args.server)
            except Exception as e:
                logger.error(f"Client failed: {str(e)}")

2025-05-10 12:24:38,319 - FraminghamClient - INFO - Using device: cpu
2025-05-10 12:24:38,330 - FraminghamClient - INFO - Loaded framingham_part1.csv with shape (1060, 16)
2025-05-10 12:24:38,334 - FraminghamClient - INFO - Class distribution: {0: 884, 1: 176}
2025-05-10 12:24:38,346 - FraminghamClient - INFO - Created dataloader with 1060 samples and 15 features
2025-05-10 12:24:38,351 - FraminghamClient - INFO - Model initialized with input size: 15
2025-05-10 12:24:38,351 - FraminghamClient - INFO - Initialized client with device: cpu
2025-05-10 12:24:38,351 - FraminghamClient - INFO - Starting client 0 and connecting to localhost:8080
	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions

Running in Jupyter/IPython environment

===== Framingham Heart Study FL Client 0 =====
Server:        localhost:8080
Data file:     framingham_part1.csv
Local epochs:  3
Batch size:    32
Device:        cpu

Connecting to server...



INFO :      
2025-05-10 12:24:39,516 - flwr - INFO - 
INFO :      Received: train message abcf9731-88cc-4dc1-b250-3a0fe157b41c
2025-05-10 12:24:39,518 - flwr - INFO - Received: train message abcf9731-88cc-4dc1-b250-3a0fe157b41c
2025-05-10 12:24:39,526 - flwr - WARNING - Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
2025-05-10 12:24:39,534 - FraminghamClient - INFO - Parameters updated from server
2025-05-10 12:24:39,548 - FraminghamClient - INFO - Epoch 1/3 - Batch 0/34 - Loss: 0.7681
2025-05-10 12:24:39,598 - FraminghamClient - INFO - Epoch 1/3 - Batch 5/34 - Loss: 0.7268
2025-05-10 12:24:39,655 - FraminghamClient - INFO - Epoch 1/3 - Batch 10/34 - Loss: 0.7003
2025-05-10 12:24:39,703 - FraminghamClient - INFO - Epoch 1/3 - Batch 15/34 - Loss: 0.6890
2025-05-10 12:24:39,765 - FraminghamClient - INFO - Epoch 1/3 - Batch 20/34 - Loss:

fed prox


In [14]:
import torch
import torch.nn as nn
import flwr as fl
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
import os
import sys
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger("FraminghamClient")

# Client configuration
CLIENT_CONFIG = {
    "local_epochs": 3,
    "batch_size": 32,
    "learning_rate": 0.001,
    "weight_decay": 1e-5,
    "dropout_rate": 0.3,
    "server_address": "localhost:8080",
    "proximal_mu": 0.01  # FedProx hyperparameter (controls the proximal term strength)
}

# Model for Framingham Heart Study data
class HeartDiseaseModel(nn.Module):
    def __init__(self, input_size):
        super(HeartDiseaseModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Dropout(CLIENT_CONFIG["dropout_rate"]),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(CLIENT_CONFIG["dropout_rate"]),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

# FedProx Loss Function
class FedProxLoss(nn.Module):
    def __init__(self, base_criterion, mu=0.01):
        super(FedProxLoss, self).__init__()
        self.base_criterion = base_criterion
        self.mu = mu  # Proximal term coefficient
        
    def forward(self, y_pred, y_true, model_params, global_params):
        # Calculate the base loss (e.g., BCE loss)
        base_loss = self.base_criterion(y_pred, y_true)
        
        # Calculate the proximal term if global parameters are provided
        proximal_term = 0.0
        if global_params is not None:
            # Sum up the squared L2 norm of the difference between local and global model parameters
            for local_param, global_param in zip(model_params, global_params):
                proximal_term += torch.sum((local_param - global_param) ** 2)
                
            # Add the weighted proximal term to the base loss
            loss = base_loss + (self.mu / 2) * proximal_term
            return loss
        
        # If no global parameters are provided, just return the base loss
        return base_loss

# Load and preprocess Framingham data
def load_data(data_path):
    """Load and preprocess Framingham Heart Study data"""
    try:
        # Read CSV data
        df = pd.read_csv(data_path)
        logger.info(f"Loaded {data_path} with shape {df.shape}")
        
        # Check for missing values
        missing_values = df.isnull().sum().sum()
        if missing_values > 0:
            logger.info(f"Found {missing_values} missing values, dropping rows with missing values")
            df.dropna(inplace=True)
            logger.info(f"Shape after dropping missing values: {df.shape}")
        
        # Ensure the target column exists
        if "TenYearCHD" not in df.columns:
            raise ValueError("Target column 'TenYearCHD' not found in dataset!")
            
        # Split features and target
        X = df.drop(columns=["TenYearCHD"])
        y = df["TenYearCHD"]
        
        # Show class distribution
        logger.info(f"Class distribution: {y.value_counts().to_dict()}")
        
        # Standardize features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # Convert to tensors
        X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
        y_tensor = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)
        
        # Create dataset and dataloader
        dataset = TensorDataset(X_tensor, y_tensor)
        dataloader = DataLoader(dataset, batch_size=CLIENT_CONFIG["batch_size"], shuffle=True)
        
        logger.info(f"Created dataloader with {len(dataset)} samples and {X.shape[1]} features")
        return dataloader, X.shape[1]
    
    except Exception as e:
        logger.error(f"Error loading data: {str(e)}")
        raise

# Client class for Federated Learning with FedProx
class FraminghamClient(fl.client.NumPyClient):
    def __init__(self, model, dataloader, device):
        self.model = model
        self.dataloader = dataloader
        self.device = device
        self.global_params = None  # Store global model parameters for FedProx
        logger.info(f"Initialized client with device: {device}")
        
    def get_parameters(self, config):
        """Get model parameters as a list of NumPy arrays"""
        # Using detach() to prevent gradient error
        return [val.detach().cpu().numpy() for val in self.model.parameters()]
    
    def set_parameters(self, parameters):
        """Set model parameters from a list of NumPy arrays"""
        # Convert to torch tensors
        self.global_params = [torch.tensor(p, device=self.device) for p in parameters]
        
        # Update model
        params_dict = zip(self.model.state_dict().keys(), parameters)
        state_dict = {k: torch.tensor(v, device=self.device) for k, v in params_dict}
        self.model.load_state_dict(state_dict, strict=True)
        logger.info("Parameters updated from server")
        
    def fit(self, parameters, config):
        """Train the model on local data with FedProx"""
        # Update model with server parameters
        self.set_parameters(parameters)
        
        # Train the model
        self.model.train()
        
        # Standard loss function
        criterion = nn.BCELoss()
        
        # FedProx loss function
        proximal_criterion = FedProxLoss(criterion, mu=CLIENT_CONFIG["proximal_mu"])
        
        optimizer = torch.optim.Adam(
            self.model.parameters(), 
            lr=CLIENT_CONFIG["learning_rate"],
            weight_decay=CLIENT_CONFIG["weight_decay"]
        )
        
        # Metrics for tracking
        total_loss = 0.0
        total_samples = 0
        correct = 0
        
        # Train for multiple epochs
        for epoch in range(CLIENT_CONFIG["local_epochs"]):
            epoch_loss = 0.0
            epoch_samples = 0
            
            for batch_idx, (X, y) in enumerate(self.dataloader):
                # Move tensors to device
                X, y = X.to(self.device), y.to(self.device)
                
                # Forward pass
                y_pred = self.model(X)
                
                # Calculate loss with proximal term
                loss = proximal_criterion(
                    y_pred, 
                    y, 
                    self.model.parameters(),  # Current model parameters
                    self.global_params        # Global model parameters
                )
                
                # Backward pass
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                # Update metrics
                batch_loss = loss.item() * X.size(0)
                total_loss += batch_loss
                epoch_loss += batch_loss
                total_samples += X.size(0)
                epoch_samples += X.size(0)
                
                # Calculate accuracy
                predicted = (y_pred > 0.5).float()
                correct += (predicted == y).sum().item()
                
                # Log progress occasionally
                if batch_idx % 5 == 0:
                    logger.info(
                        f"Epoch {epoch+1}/{CLIENT_CONFIG['local_epochs']} - "
                        f"Batch {batch_idx}/{len(self.dataloader)} - "
                        f"Loss: {loss.item():.4f}"
                    )
            
            # Log epoch metrics
            logger.info(
                f"Epoch {epoch+1}/{CLIENT_CONFIG['local_epochs']} completed - "
                f"Loss: {epoch_loss/epoch_samples:.4f}"
            )
        
        # Calculate final metrics
        avg_loss = total_loss / total_samples if total_samples > 0 else 0
        accuracy = correct / total_samples if total_samples > 0 else 0
        
        logger.info(f"Training completed - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")
        
        # Return updated model parameters and metrics
        return self.get_parameters({}), total_samples, {"loss": float(avg_loss), "accuracy": float(accuracy)}
    
    def evaluate(self, parameters, config):
        """Evaluate the model on local data"""
        # Update model with server parameters
        self.set_parameters(parameters)
        
        # Evaluate the model
        self.model.eval()
        criterion = nn.BCELoss()
        
        loss = 0.0
        total = 0
        correct = 0
        
        with torch.no_grad():
            for X, y in self.dataloader:
                # Move tensors to device
                X, y = X.to(self.device), y.to(self.device)
                
                # Forward pass
                y_pred = self.model(X)
                batch_loss = criterion(y_pred, y).item()
                
                # Update metrics
                loss += batch_loss * X.size(0)
                total += X.size(0)
                
                # Calculate accuracy
                predicted = (y_pred > 0.5).float()
                correct += (predicted == y).sum().item()
        
        # Calculate final metrics
        avg_loss = loss / total if total > 0 else 0
        accuracy = correct / total if total > 0 else 0
        
        logger.info(f"Evaluation - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")
        
        # Return metrics
        return float(avg_loss), total, {"accuracy": float(accuracy)}

def start_client(client_id=0, server_address=None):
    """Initialize and start a client"""
    # Update server address if provided
    if server_address:
        CLIENT_CONFIG["server_address"] = server_address
    
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logger.info(f"Using device: {device}")
    
    # Determine which data file to use based on client ID
    data_path = f"framingham_part{client_id+1}.csv"
    
    # Try alternative naming if file doesn't exist
    if not os.path.exists(data_path):
        alternative_path = f"framingham_part{client_id+1}.csv"
        if os.path.exists(alternative_path):
            data_path = alternative_path
        else:
            logger.error(f"Data file {data_path} not found")
            return
    
    # Load data
    dataloader, input_size = load_data(data_path)
    
    # Initialize model
    model = HeartDiseaseModel(input_size=input_size).to(device)
    logger.info(f"Model initialized with input size: {input_size}")
    
    # Create client
    client = FraminghamClient(model, dataloader, device)
    
    # Start client
    logger.info(f"Starting client {client_id} and connecting to {CLIENT_CONFIG['server_address']}")
    
    print(f"\n===== Framingham Heart Study FL Client {client_id} (FedProx) =====")
    print(f"Server:        {CLIENT_CONFIG['server_address']}")
    print(f"Data file:     {data_path}")
    print(f"Local epochs:  {CLIENT_CONFIG['local_epochs']}")
    print(f"Batch size:    {CLIENT_CONFIG['batch_size']}")
    print(f"Proximal mu:   {CLIENT_CONFIG['proximal_mu']}")
    print(f"Device:        {device}")
    print("=================================================")
    print(f"\nConnecting to server...\n")
    
    fl.client.start_client(server_address=CLIENT_CONFIG["server_address"], client=client)

# For Jupyter usage
if __name__ == "__main__":
    # Check if running in Jupyter
    if 'ipykernel' in sys.modules:
        print("Running in Jupyter/IPython environment")
        # Default to client ID 0, can be changed by user
        start_client(client_id=0)
    else:
        # For command line use
        import argparse
        parser = argparse.ArgumentParser(description="Framingham Heart Study FL Client")
        parser.add_argument("--id", type=int, default=0, help="Client ID (0, 1, or 2)")
        parser.add_argument("--server", type=str, default="localhost:8080", help="Server address")
        parser.add_argument("--mu", type=float, default=0.01, help="FedProx proximal term strength")
        
        args = parser.parse_args()
        
        if args.id not in [0, 1, 2]:
            logger.error("Client ID must be 0, 1, or 2")
        else:
            try:
                # Set FedProx hyperparameter
                CLIENT_CONFIG["proximal_mu"] = args.mu
                
                start_client(args.id, args.server)
            except Exception as e:
                logger.error(f"Client failed: {str(e)}")

2025-05-11 15:01:10,436 - FraminghamClient - INFO - Using device: cpu
2025-05-11 15:01:10,452 - FraminghamClient - INFO - Loaded framingham_part1.csv with shape (1060, 16)
2025-05-11 15:01:10,460 - FraminghamClient - INFO - Class distribution: {0: 884, 1: 176}
2025-05-11 15:01:10,468 - FraminghamClient - INFO - Created dataloader with 1060 samples and 15 features
2025-05-11 15:01:10,468 - FraminghamClient - INFO - Model initialized with input size: 15
2025-05-11 15:01:10,476 - FraminghamClient - INFO - Initialized client with device: cpu
2025-05-11 15:01:10,476 - FraminghamClient - INFO - Starting client 0 and connecting to localhost:8080
	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions

Running in Jupyter/IPython environment

===== Framingham Heart Study FL Client 0 (FedProx) =====
Server:        localhost:8080
Data file:     framingham_part1.csv
Local epochs:  3
Batch size:    32
Proximal mu:   0.01
Device:        cpu

Connecting to server...



2025-05-11 15:01:10,663 - FraminghamClient - INFO - Epoch 1/3 - Batch 5/34 - Loss: 0.6822
2025-05-11 15:01:10,737 - FraminghamClient - INFO - Epoch 1/3 - Batch 10/34 - Loss: 0.6390
2025-05-11 15:01:10,802 - FraminghamClient - INFO - Epoch 1/3 - Batch 15/34 - Loss: 0.6277
2025-05-11 15:01:10,864 - FraminghamClient - INFO - Epoch 1/3 - Batch 20/34 - Loss: 0.5858
2025-05-11 15:01:10,927 - FraminghamClient - INFO - Epoch 1/3 - Batch 25/34 - Loss: 0.5863
2025-05-11 15:01:11,010 - FraminghamClient - INFO - Epoch 1/3 - Batch 30/34 - Loss: 0.5035
2025-05-11 15:01:11,120 - FraminghamClient - INFO - Epoch 1/3 completed - Loss: 0.6269
2025-05-11 15:01:11,308 - FraminghamClient - INFO - Epoch 2/3 - Batch 0/34 - Loss: 0.5081
2025-05-11 15:01:11,406 - FraminghamClient - INFO - Epoch 2/3 - Batch 5/34 - Loss: 0.5570
2025-05-11 15:01:11,580 - FraminghamClient - INFO - Epoch 2/3 - Batch 10/34 - Loss: 0.5547
2025-05-11 15:01:11,646 - FraminghamClient - INFO - Epoch 2/3 - Batch 15/34 - Loss: 0.3674
2025-0